# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [1]:
#Executar no notebook:
#!pip install findspark
#!pip install requests
#!pip install unidecode
#!pip install pyspark
#!pip install pandas

In [2]:
# Executar na linha de comando:

# instalando java
# sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# Fazendo download do spark 
# sudo wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
# tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/spark-3.1.2-bin-hadoop2.7"

In [5]:
#Criando a sessão do pyspark

import findspark
findspark.init()

import requests

import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[*]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

spark

23/01/23 10:00:33 WARN Utils: Your hostname, DESKTOP-A27L3NC resolves to a loopback address: 127.0.1.1; using 172.23.138.162 instead (on interface eth0)
23/01/23 10:00:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/23 10:00:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
# Customizando um contexto SSL para conectar com conexões SSL legadas
# Quando a API é antiga é preciso usar esse artifício.

import urllib3
import ssl

class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # Criando um contexto SSL customizado.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

# Buscar cidades do Vale do Paraíba

In [7]:
# Buscar cidades no site do IBGE

url = 'https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios'
response = get_legacy_session().get(url)
data = response.json()

df_cidades = spark.read.json(spark.sparkContext.parallelize([data]))
df_cidades = df_cidades.withColumnRenamed('regiao-imediata', 'regiao_imediata')
df_cidades = df_cidades.selectExpr("id", 'nome',"regiao_imediata['nome'] as regiao")
df_cidades.show(truncate=False)
print('Quantidade de cidades: ', df_cidades.count())

+-------+------------------+---------------------------------------+
|id     |nome              |regiao                                 |
+-------+------------------+---------------------------------------+
|3502507|Aparecida         |Guaratinguetá                          |
|3503158|Arapeí            |Cruzeiro                               |
|3503505|Areias            |Cruzeiro                               |
|3504909|Bananal           |Cruzeiro                               |
|3508504|Caçapava          |São José dos Campos                    |
|3508603|Cachoeira Paulista|Cruzeiro                               |
|3509700|Campos do Jordão  |Taubaté - Pindamonhangaba              |
|3509957|Canas             |Guaratinguetá                          |
|3510500|Caraguatatuba     |Caraguatatuba - Ubatuba - São Sebastião|
|3513405|Cruzeiro          |Cruzeiro                               |
|3513603|Cunha             |Guaratinguetá                          |
|3518404|Guaratinguetá     |Guarat

# Criar data frame com as cidades

In [8]:
# Buscar cidades no site do ACCUWEATHER

import requests

from pyspark.sql.types import StructField, StructType, StringType, BooleanType, FloatType

def consultaCidadesAccuweather(countrycode, api_key, city_name):
    url = f"http://dataservice.accuweather.com/locations/v1/cities/{countrycode}/search?apikey={api_key}&q={city_name}"
    response = requests.get(url)
    cidade_json = response.json()
    return cidade_json

In [10]:
from pyspark.sql.functions import lit

#API KEY disponível ao se registrar em https://developer.accuweather.com/
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

countrycode = 'br'

schema = StructType([
        
                StructField("Key", StringType(), True),
        
                StructField("LocalizedName", StringType(), True),
                    
                StructField("Country", StructType([
                    StructField("LocalizedName", StringType(), True)
                ])),
        
                StructField("AdministrativeArea", StructType([
                    StructField("LocalizedName", StringType(), True)
                ])),
        
                StructField("GeoPosition", StructType([
                    StructField("Latitude", FloatType(), True),
                    StructField("Longitude", FloatType(), True)
                ]))
                   
            ])

primeiraVez = True

for elemento in df_cidades.collect():
    
    cidades_json = consultaCidadesAccuweather(countrycode, api_key, elemento['nome'])
        
    print(cidades_json)
    
    if type(cidades_json) is not dict:
         
        df_auxiliar = spark.createDataFrame(cidades_json, schema)
        df_auxiliar = df_auxiliar.withColumn("CodigoDaCidade", lit(elemento['id']))
        
        
        if primeiraVez:
            df_cidades_accuweather = df_auxiliar
            primeiraVez = False
        else:
            df_cidades_accuweather = df_cidades_accuweather.union(df_auxiliar)
    
    else:
        print("Comportamento inesperado. Retorno do site: \n")
        print(cidades_json,"\n")
        break
            
if not primeiraVez:
    df_cidades_accuweather_backup = df_cidades_accuweather
    df_cidades_accuweather.printSchema()         

[{'Version': 1, 'Key': '2731691', 'Type': 'City', 'Rank': 55, 'LocalizedName': 'Aparecida', 'EnglishName': 'Aparecida', 'PrimaryPostalCode': '', 'Region': {'ID': 'SAM', 'LocalizedName': 'South America', 'EnglishName': 'South America'}, 'Country': {'ID': 'BR', 'LocalizedName': 'Brazil', 'EnglishName': 'Brazil'}, 'AdministrativeArea': {'ID': 'SP', 'LocalizedName': 'São Paulo', 'EnglishName': 'São Paulo', 'Level': 1, 'LocalizedType': 'State', 'EnglishType': 'State', 'CountryID': 'BR'}, 'TimeZone': {'Code': 'BRT', 'Name': 'America/Sao_Paulo', 'GmtOffset': -3.0, 'IsDaylightSaving': False, 'NextOffsetChange': None}, 'GeoPosition': {'Latitude': -23.977, 'Longitude': -46.308, 'Elevation': {'Metric': {'Value': 1.0, 'Unit': 'm', 'UnitType': 5}, 'Imperial': {'Value': 3.0, 'Unit': 'ft', 'UnitType': 0}}}, 'IsAlias': False, 'ParentCity': {'Key': '36358', 'LocalizedName': 'Santos', 'EnglishName': 'Santos'}, 'SupplementalAdminAreas': [{'Level': 2, 'LocalizedName': 'Santos', 'EnglishName': 'Santos'}, {

[{'Version': 1, 'Key': '36422', 'Type': 'City', 'Rank': 85, 'LocalizedName': 'Arapeí', 'EnglishName': 'Arapeí', 'PrimaryPostalCode': '', 'Region': {'ID': 'SAM', 'LocalizedName': 'South America', 'EnglishName': 'South America'}, 'Country': {'ID': 'BR', 'LocalizedName': 'Brazil', 'EnglishName': 'Brazil'}, 'AdministrativeArea': {'ID': 'SP', 'LocalizedName': 'São Paulo', 'EnglishName': 'São Paulo', 'Level': 1, 'LocalizedType': 'State', 'EnglishType': 'State', 'CountryID': 'BR'}, 'TimeZone': {'Code': 'BRT', 'Name': 'America/Sao_Paulo', 'GmtOffset': -3.0, 'IsDaylightSaving': False, 'NextOffsetChange': None}, 'GeoPosition': {'Latitude': -22.675, 'Longitude': -44.448, 'Elevation': {'Metric': {'Value': 588.0, 'Unit': 'm', 'UnitType': 5}, 'Imperial': {'Value': 1928.0, 'Unit': 'ft', 'UnitType': 0}}}, 'IsAlias': False, 'SupplementalAdminAreas': [{'Level': 2, 'LocalizedName': 'Arapeí', 'EnglishName': 'Arapeí'}, {'Level': 3, 'LocalizedName': 'Arapeí', 'EnglishName': 'Arapeí'}], 'DataSets': ['AirQual

[{'Version': 1, 'Key': '45836', 'Type': 'City', 'Rank': 55, 'LocalizedName': 'Caçapava', 'EnglishName': 'Caçapava', 'PrimaryPostalCode': '', 'Region': {'ID': 'SAM', 'LocalizedName': 'South America', 'EnglishName': 'South America'}, 'Country': {'ID': 'BR', 'LocalizedName': 'Brazil', 'EnglishName': 'Brazil'}, 'AdministrativeArea': {'ID': 'SP', 'LocalizedName': 'São Paulo', 'EnglishName': 'São Paulo', 'Level': 1, 'LocalizedType': 'State', 'EnglishType': 'State', 'CountryID': 'BR'}, 'TimeZone': {'Code': 'BRT', 'Name': 'America/Sao_Paulo', 'GmtOffset': -3.0, 'IsDaylightSaving': False, 'NextOffsetChange': None}, 'GeoPosition': {'Latitude': -23.101, 'Longitude': -45.708, 'Elevation': {'Metric': {'Value': 576.0, 'Unit': 'm', 'UnitType': 5}, 'Imperial': {'Value': 1889.0, 'Unit': 'ft', 'UnitType': 0}}}, 'IsAlias': False, 'SupplementalAdminAreas': [{'Level': 2, 'LocalizedName': 'Caçapava', 'EnglishName': 'Caçapava'}, {'Level': 3, 'LocalizedName': 'Caçapava', 'EnglishName': 'Caçapava'}], 'DataSets

[{'Version': 1, 'Key': '36341', 'Type': 'City', 'Rank': 45, 'LocalizedName': 'Guaratinguetá', 'EnglishName': 'Guaratinguetá', 'PrimaryPostalCode': '', 'Region': {'ID': 'SAM', 'LocalizedName': 'South America', 'EnglishName': 'South America'}, 'Country': {'ID': 'BR', 'LocalizedName': 'Brazil', 'EnglishName': 'Brazil'}, 'AdministrativeArea': {'ID': 'SP', 'LocalizedName': 'São Paulo', 'EnglishName': 'São Paulo', 'Level': 1, 'LocalizedType': 'State', 'EnglishType': 'State', 'CountryID': 'BR'}, 'TimeZone': {'Code': 'BRT', 'Name': 'America/Sao_Paulo', 'GmtOffset': -3.0, 'IsDaylightSaving': False, 'NextOffsetChange': None}, 'GeoPosition': {'Latitude': -22.817, 'Longitude': -45.192, 'Elevation': {'Metric': {'Value': 520.0, 'Unit': 'm', 'UnitType': 5}, 'Imperial': {'Value': 1705.0, 'Unit': 'ft', 'UnitType': 0}}}, 'IsAlias': False, 'SupplementalAdminAreas': [{'Level': 2, 'LocalizedName': 'Guaratinguetá', 'EnglishName': 'Guaratinguetá'}, {'Level': 3, 'LocalizedName': 'Guaratinguetá', 'EnglishName'

[{'Version': 1, 'Key': '41619', 'Type': 'City', 'Rank': 85, 'LocalizedName': 'Monteiro Lobato', 'EnglishName': 'Monteiro Lobato', 'PrimaryPostalCode': '', 'Region': {'ID': 'SAM', 'LocalizedName': 'South America', 'EnglishName': 'South America'}, 'Country': {'ID': 'BR', 'LocalizedName': 'Brazil', 'EnglishName': 'Brazil'}, 'AdministrativeArea': {'ID': 'SP', 'LocalizedName': 'São Paulo', 'EnglishName': 'São Paulo', 'Level': 1, 'LocalizedType': 'State', 'EnglishType': 'State', 'CountryID': 'BR'}, 'TimeZone': {'Code': 'BRT', 'Name': 'America/Sao_Paulo', 'GmtOffset': -3.0, 'IsDaylightSaving': False, 'NextOffsetChange': None}, 'GeoPosition': {'Latitude': -22.955, 'Longitude': -45.848, 'Elevation': {'Metric': {'Value': 903.0, 'Unit': 'm', 'UnitType': 5}, 'Imperial': {'Value': 2961.0, 'Unit': 'ft', 'UnitType': 0}}}, 'IsAlias': False, 'SupplementalAdminAreas': [{'Level': 2, 'LocalizedName': 'Monteiro Lobato', 'EnglishName': 'Monteiro Lobato'}, {'Level': 3, 'LocalizedName': 'Monteiro Lobato', 'En

[{'Version': 1, 'Key': '41449', 'Type': 'City', 'Rank': 75, 'LocalizedName': 'Roseira', 'EnglishName': 'Roseira', 'PrimaryPostalCode': '', 'Region': {'ID': 'SAM', 'LocalizedName': 'South America', 'EnglishName': 'South America'}, 'Country': {'ID': 'BR', 'LocalizedName': 'Brazil', 'EnglishName': 'Brazil'}, 'AdministrativeArea': {'ID': 'SP', 'LocalizedName': 'São Paulo', 'EnglishName': 'São Paulo', 'Level': 1, 'LocalizedType': 'State', 'EnglishType': 'State', 'CountryID': 'BR'}, 'TimeZone': {'Code': 'BRT', 'Name': 'America/Sao_Paulo', 'GmtOffset': -3.0, 'IsDaylightSaving': False, 'NextOffsetChange': None}, 'GeoPosition': {'Latitude': -22.897, 'Longitude': -45.309, 'Elevation': {'Metric': {'Value': 567.0, 'Unit': 'm', 'UnitType': 5}, 'Imperial': {'Value': 1859.0, 'Unit': 'ft', 'UnitType': 0}}}, 'IsAlias': False, 'SupplementalAdminAreas': [{'Level': 2, 'LocalizedName': 'Roseira', 'EnglishName': 'Roseira'}, {'Level': 3, 'LocalizedName': 'Roseira', 'EnglishName': 'Roseira'}], 'DataSets': ['A

[{'Version': 1, 'Key': '36957', 'Type': 'City', 'Rank': 65, 'LocalizedName': 'São Sebastião', 'EnglishName': 'São Sebastião', 'PrimaryPostalCode': '', 'Region': {'ID': 'SAM', 'LocalizedName': 'South America', 'EnglishName': 'South America'}, 'Country': {'ID': 'BR', 'LocalizedName': 'Brazil', 'EnglishName': 'Brazil'}, 'AdministrativeArea': {'ID': 'AL', 'LocalizedName': 'Alagoas', 'EnglishName': 'Alagoas', 'Level': 1, 'LocalizedType': 'State', 'EnglishType': 'State', 'CountryID': 'BR'}, 'TimeZone': {'Code': 'BRT', 'Name': 'America/Maceio', 'GmtOffset': -3.0, 'IsDaylightSaving': False, 'NextOffsetChange': None}, 'GeoPosition': {'Latitude': -9.932, 'Longitude': -36.551, 'Elevation': {'Metric': {'Value': 213.0, 'Unit': 'm', 'UnitType': 5}, 'Imperial': {'Value': 698.0, 'Unit': 'ft', 'UnitType': 0}}}, 'IsAlias': False, 'SupplementalAdminAreas': [{'Level': 2, 'LocalizedName': 'São Sebastião', 'EnglishName': 'São Sebastião'}, {'Level': 3, 'LocalizedName': 'São Sebastião', 'EnglishName': 'São Se

root
 |-- Key: string (nullable = true)
 |-- LocalizedName: string (nullable = true)
 |-- Country: struct (nullable = true)
 |    |-- LocalizedName: string (nullable = true)
 |-- AdministrativeArea: struct (nullable = true)
 |    |-- LocalizedName: string (nullable = true)
 |-- GeoPosition: struct (nullable = true)
 |    |-- Latitude: float (nullable = true)
 |    |-- Longitude: float (nullable = true)
 |-- CodigoDaCidade: integer (nullable = false)



In [11]:
# Selecionando apenas as colunas desejadas do dataframe

df_cidades_accuweather = df_cidades_accuweather.selectExpr(
    "Key as Location_key",
    "CodigoDaCidade as CodigoDaCidade", 
    "LocalizedName as Cidade",
    "Country['LocalizedName'] as Pais",
    "AdministrativeArea['LocalizedName'] as Regiao",
    "GeoPosition['Latitude'] as Latitude",
    "GeoPosition['Longitude'] as Longitude")


df_cidades_accuweather = df_cidades_accuweather.where("regiao == 'São Paulo'")
df_cidades_accuweather = df_cidades_accuweather.dropDuplicates(subset=['Cidade'])

print('Quantidade de cidades: ', df_cidades_accuweather.count())
df_cidades_accuweather.show(55,truncate=False)

+------------+--------------+-----------------------+------+---------+--------+---------+
|Location_key|CodigoDaCidade|Cidade                 |Pais  |Regiao   |Latitude|Longitude|
+------------+--------------+-----------------------+------+---------+--------+---------+
|41491       |3548609       |São Bento do Sapucaí   |Brazil|São Paulo|-22.685 |-45.737  |
|41623       |3535606       |Paraibuna              |Brazil|São Paulo|-23.386 |-45.663  |
|2731691     |3502507       |Aparecida              |Brazil|São Paulo|-23.977 |-46.308  |
|36614       |3520202       |Igaratá                |Brazil|São Paulo|-23.205 |-46.156  |
|36422       |3503158       |Arapeí                 |Brazil|São Paulo|-22.675 |-44.448  |
|41462       |3546009       |Santa Branca           |Brazil|São Paulo|-23.396 |-45.888  |
|41398       |3538006       |Pindamonhangaba        |Brazil|São Paulo|-22.927 |-45.462  |
|45837       |3508603       |Cachoeira Paulista     |Brazil|São Paulo|-22.665 |-45.011  |
|36679    

Quantidade de cidades:  39


# Criar view com as cidades

In [12]:
# Criar view com as cidades
df_cidades_accuweather.createOrReplaceTempView("cities_temp_view")

# Buscar previsão do tempo para as cidades

In [13]:
# Buscar previsão do tempo para as cidades

import requests

from pyspark.sql.types import StructField, StructType, StringType, BooleanType, FloatType

def consultaPrevisao(location_key, api_key):
    weather_url = f"http://dataservice.accuweather.com/forecasts/v1/daily/5day/{location_key}?apikey={api_key}"
    response = requests.get(weather_url)
    previsao_json = response.json()
    return previsao_json

# Criar data frame com as previsões

In [15]:
# Transformando os Json em um dataframe  

#API KEY disponível ao se registrar em https://developer.accuweather.com/
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

schema = StructType([
        
                StructField("Date", StringType(), True),
                
                StructField("Temperature", StructType([
                    StructField("Maximum", StructType([
                        StructField("Value", FloatType(), True)
                    ])),
                    StructField("Minimum", StructType([
                        StructField("Value", FloatType(), True)
                    ]))
                ])),
        
                StructField("Day", StructType([
                    StructField("HasPrecipitation", BooleanType(), True),
                    StructField("IconPhrase", StringType(), True)
                ])),
        
                StructField("Night", StructType([
                    StructField("HasPrecipitation", BooleanType(), True),
                    StructField("IconPhrase", StringType(), True)
                ]))
                   
            ])

primeiraVez = True

for elemento in df_cidades_accuweather.collect():
    
    previsao_json = consultaPrevisao(elemento['Location_key'], api_key)
    
    print(previsao_json)
    
    if 'DailyForecasts'in previsao_json:
       
        df_auxiliar = spark.createDataFrame(previsao_json['DailyForecasts'], schema)
        df_auxiliar = df_auxiliar.withColumn("Location_key", lit(elemento['Location_key']))
                   
        if primeiraVez:
            df_weather = df_auxiliar
            primeiraVez = False
        else:
            df_weather = df_weather.union(df_auxiliar)
    
    else:
        print("Comportamento inesperado. Retorno do site: \n")
        print(previsao_json)
        break
        
if not primeiraVez: 
    df_weather_backup = df_weather
    df_weather.printSchema() 

{'Headline': {'EffectiveDate': '2023-01-19T07:00:00-03:00', 'EffectiveEpochDate': 1674122400, 'Severity': 5, 'Text': 'A thunderstorm today', 'Category': 'thunderstorm', 'EndDate': '2023-01-19T19:00:00-03:00', 'EndEpochDate': 1674165600, 'MobileLink': 'http://www.accuweather.com/en/br/sao-bento-do-sapucai/41491/daily-weather-forecast/41491?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/sao-bento-do-sapucai/41491/daily-weather-forecast/41491?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 62.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 76.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 15, 'IconPhrase': 'Thunderstorms', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Night': {'Icon': 38, 'IconPhrase': 'Mostly cloudy', 'HasPrecipitation': False}, 'Sources': ['AccuWeather'], 'MobileLink': 'http://www.accuweather.com/en/br/sao-bento-do-sapuca

{'Headline': {'EffectiveDate': '2023-01-19T13:00:00-03:00', 'EffectiveEpochDate': 1674144000, 'Severity': 5, 'Text': 'A thundershower this afternoon', 'Category': 'thunderstorm', 'EndDate': '2023-01-19T19:00:00-03:00', 'EndEpochDate': 1674165600, 'MobileLink': 'http://www.accuweather.com/en/br/santa-branca/41462/daily-weather-forecast/41462?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/santa-branca/41462/daily-weather-forecast/41462?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 66.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 84.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 12, 'IconPhrase': 'Showers', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Night': {'Icon': 38, 'IconPhrase': 'Mostly cloudy', 'HasPrecipitation': False}, 'Sources': ['AccuWeather'], 'MobileLink': 'http://www.accuweather.com/en/br/santa-branca/41462/daily-weathe

{'Headline': {'EffectiveDate': '2023-01-19T13:00:00-03:00', 'EffectiveEpochDate': 1674144000, 'Severity': 5, 'Text': 'Showers and a thunderstorm this afternoon through this evening', 'Category': 'thunderstorm', 'EndDate': '2023-01-20T01:00:00-03:00', 'EndEpochDate': 1674187200, 'MobileLink': 'http://www.accuweather.com/en/br/cruzeiro/41669/daily-weather-forecast/41669?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/cruzeiro/41669/daily-weather-forecast/41669?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 66.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 81.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 12, 'IconPhrase': 'Showers', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Night': {'Icon': 12, 'IconPhrase': 'Showers', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Sources': ['AccuWeathe

{'Headline': {'EffectiveDate': '2023-01-19T13:00:00-03:00', 'EffectiveEpochDate': 1674144000, 'Severity': 5, 'Text': 'A thundershower this afternoon', 'Category': 'thunderstorm', 'EndDate': '2023-01-19T19:00:00-03:00', 'EndEpochDate': 1674165600, 'MobileLink': 'http://www.accuweather.com/en/br/bananal/2730800/daily-weather-forecast/2730800?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/bananal/2730800/daily-weather-forecast/2730800?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 67.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 81.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 12, 'IconPhrase': 'Showers', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Light'}, 'Night': {'Icon': 38, 'IconPhrase': 'Mostly cloudy', 'HasPrecipitation': False}, 'Sources': ['AccuWeather'], 'MobileLink': 'http://www.accuweather.com/en/br/bananal/2730800/daily-weather-foreca

{'Headline': {'EffectiveDate': '2023-01-23T13:00:00-03:00', 'EffectiveEpochDate': 1674489600, 'Severity': 2, 'Text': 'Thunderstorms, some heavy, Monday afternoon', 'Category': 'thunderstorm', 'EndDate': '2023-01-23T19:00:00-03:00', 'EndEpochDate': 1674511200, 'MobileLink': 'http://www.accuweather.com/en/br/monteiro-lobato/41619/daily-weather-forecast/41619?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/monteiro-lobato/41619/daily-weather-forecast/41619?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 64.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 78.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 15, 'IconPhrase': 'Thunderstorms', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Night': {'Icon': 38, 'IconPhrase': 'Mostly cloudy', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Light'}, 'Sources': ['AccuWe

{'Headline': {'EffectiveDate': '2023-01-22T07:00:00-03:00', 'EffectiveEpochDate': 1674381600, 'Severity': 4, 'Text': 'Pleasant Sunday', 'Category': 'mild', 'EndDate': None, 'EndEpochDate': None, 'MobileLink': 'http://www.accuweather.com/en/br/guaratingueta/36341/daily-weather-forecast/36341?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/guaratingueta/36341/daily-weather-forecast/36341?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 67.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 84.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 12, 'IconPhrase': 'Showers', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Night': {'Icon': 42, 'IconPhrase': 'Mostly cloudy w/ t-storms', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Sources': ['AccuWeather'], 'MobileLink': 'http://www.accuweather.com/en/br/gua

{'Headline': {'EffectiveDate': '2023-01-19T13:00:00-03:00', 'EffectiveEpochDate': 1674144000, 'Severity': 5, 'Text': 'A thundershower this afternoon', 'Category': 'thunderstorm', 'EndDate': '2023-01-19T19:00:00-03:00', 'EndEpochDate': 1674165600, 'MobileLink': 'http://www.accuweather.com/en/br/jambeiro/36653/daily-weather-forecast/36653?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/jambeiro/36653/daily-weather-forecast/36653?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 64.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 82.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 12, 'IconPhrase': 'Showers', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Night': {'Icon': 38, 'IconPhrase': 'Mostly cloudy', 'HasPrecipitation': False}, 'Sources': ['AccuWeather'], 'MobileLink': 'http://www.accuweather.com/en/br/jambeiro/36653/daily-weather-forecast/3

{'Headline': {'EffectiveDate': '2023-01-19T13:00:00-03:00', 'EffectiveEpochDate': 1674144000, 'Severity': 5, 'Text': 'A thundershower this afternoon', 'Category': 'thunderstorm', 'EndDate': '2023-01-19T19:00:00-03:00', 'EndEpochDate': 1674165600, 'MobileLink': 'http://www.accuweather.com/en/br/redencao-da-serra/41434/daily-weather-forecast/41434?lang=en-us', 'Link': 'http://www.accuweather.com/en/br/redencao-da-serra/41434/daily-weather-forecast/41434?lang=en-us'}, 'DailyForecasts': [{'Date': '2023-01-19T07:00:00-03:00', 'EpochDate': 1674122400, 'Temperature': {'Minimum': {'Value': 64.0, 'Unit': 'F', 'UnitType': 18}, 'Maximum': {'Value': 82.0, 'Unit': 'F', 'UnitType': 18}}, 'Day': {'Icon': 12, 'IconPhrase': 'Showers', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate'}, 'Night': {'Icon': 38, 'IconPhrase': 'Mostly cloudy', 'HasPrecipitation': False}, 'Sources': ['AccuWeather'], 'MobileLink': 'http://www.accuweather.com/en/br/redencao-da-serra/414

In [20]:
# Selecionando apenas as colunas desejadas do dataframe

df_weather = df_weather.selectExpr(
    "Location_key as Location_key", 
    "cast(Date as date) as Data", 
    "Temperature['Maximum']['Value'] as TemperaturaMaxima",
    "Temperature['Minimum']['Value'] as TemperaturaMinima",
    "Day['HasPrecipitation'] as VaiChoverDia",
    "Night['HasPrecipitation'] as VaiChoverNoite",
    "Day['IconPhrase'] as CondicaoDoTempoDia",
    "Night['IconPhrase'] as CondicaoDoTempoNoite")

df_weather = df_weather.withColumn("ChanceDeChuva", lit(None).cast('string')) \
                       .withColumn("NascerDoSol", lit(None).cast('string')) \
                       .withColumn("PorDoSol", lit(None).cast('string')) \
                       .withColumn("VelocidadeMaximaDoVento", lit(None).cast('string'))
                                   
                                   

#df_weather.show()
print('Quantidade de previsões: ', df_weather.count())
df_weather.show(n=5, truncate=False, vertical=True)

Quantidade de previsões:  195
-RECORD 0--------------------------------------
 Location_key            | 41491               
 Data                    | 2023-01-19          
 TemperaturaMaxima       | 76.0                
 TemperaturaMinima       | 62.0                
 VaiChoverDia            | true                
 VaiChoverNoite          | false               
 CondicaoDoTempoDia      | Thunderstorms       
 CondicaoDoTempoNoite    | Mostly cloudy       
 ChanceDeChuva           | null                
 NascerDoSol             | null                
 PorDoSol                | null                
 VelocidadeMaximaDoVento | null                
-RECORD 1--------------------------------------
 Location_key            | 41491               
 Data                    | 2023-01-20          
 TemperaturaMaxima       | 69.0                
 TemperaturaMinima       | 62.0                
 VaiChoverDia            | true                
 VaiChoverNoite          | true                
 CondicaoD

# Criar view com as previsões

In [21]:
df_weather.createOrReplaceTempView("forecast_temp_view")

# Criar DF da Tabela 1

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento

In [22]:
df_tabela1 = spark.sql(" \
                       SELECT b.Location_key, a.Cidade,a.CodigoDaCidade,b.Data, a.Regiao, a.Pais,a.Latitude,a.Longitude, \
                       b.TemperaturaMaxima,b.TemperaturaMinima, \
                       ((b.TemperaturaMaxima +b.TemperaturaMinima)/2) as TemperaturaMedia, \
                       IF((b.VaiChoverDia or b.VaiChoverNoite), 'Sim', 'Não') as VaiChover, b.ChanceDeChuva,\
                       CONCAT('Dia: ', b.CondicaoDoTempoDia, ' / Noite: ', b.CondicaoDoTempoNoite) as CondicaoDoTempo, \
                       b.NascerDoSol, b.PorDoSol, b.VelocidadeMaximaDoVento \
                       FROM cities_temp_view as A \
                       INNER JOIN forecast_temp_view as B \
                       WHERE a.location_key = b.location_key")

#df_tabela1.show(truncate=False)
print('Quantidade de previsões: ', df_tabela1.count())
df_tabela1.show(n=5, truncate=False, vertical=True)

Quantidade de previsões:  195


-RECORD 0------------------------------------------------------------------
 Location_key            | 41462                                           
 Cidade                  | Santa Branca                                    
 CodigoDaCidade          | 3546009                                         
 Data                    | 2023-01-19                                      
 Regiao                  | São Paulo                                       
 Pais                    | Brazil                                          
 Latitude                | -23.396                                         
 Longitude               | -45.888                                         
 TemperaturaMaxima       | 84.0                                            
 TemperaturaMinima       | 66.0                                            
 TemperaturaMedia        | 75.0                                            
 VaiChover               | Sim                                             
 ChanceDeChu

# Criar DF da Tabela 2

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

In [23]:
df_tabela2 = spark.sql(" \
                       SELECT b.Location_key, a.Cidade, \
                       COUNT(CASE WHEN b.VaiChoverDia or b.VaiChoverNoite THEN 1 END) AS QtdDiasVaiChover, \
                       COUNT(CASE WHEN NOT(b.VaiChoverDia or b.VaiChoverNoite) THEN 1 END) AS QtdDiasNaoVaiChover, \
                       COUNT(b.Location_key) AS TotalDiasMapeados \
                       FROM cities_temp_view as A \
                       INNER JOIN forecast_temp_view as B \
                       WHERE a.location_key = b.location_key GROUP BY  a.cidade, b.Location_key")

print('Quantidade de cidades: ', df_tabela2.count())
df_tabela2.show(truncate=False)

Quantidade de cidades:  39


+------------+-----------------------+----------------+-------------------+-----------------+
|Location_key|Cidade                 |QtdDiasVaiChover|QtdDiasNaoVaiChover|TotalDiasMapeados|
+------------+-----------------------+----------------+-------------------+-----------------+
|41462       |Santa Branca           |5               |0                  |5                |
|41623       |Paraibuna              |4               |1                  |5                |
|41628       |Piquete                |4               |1                  |5                |
|36614       |Igaratá                |5               |0                  |5                |
|41434       |Redenção da Serra      |5               |0                  |5                |
|2731691     |Aparecida              |4               |1                  |5                |
|41488       |Santo Antônio do Pinhal|5               |0                  |5                |
|41491       |São Bento do Sapucaí   |5               |0    

# Exportar CSVs

In [27]:
df_tabela1.coalesce(1).write.option("header",True).option("encoding", "ISO-8859-1") \
 .csv("tabela1.csv")

In [28]:
df_tabela2.coalesce(1).write.option("header",True).option("encoding", "ISO-8859-1") \
 .csv("tabela2.csv")